Import needed set of packages

In [ ]:
from scipy import integrate
from datetime import timedelta
import hampel
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tools import rednt_tools as tools  # custom helper tools library

Read the data from the CSV files with timestamp as index column

In [ ]:
v_df = pd.read_csv(tools.VIBRATION_FILENAME, sep=tools.SEP, header=None,
                   names=tools.V_HEADER_NAMES, index_col=[0], parse_dates=[0],
                   date_parser=tools.date_parser)

t_df = pd.read_csv(tools.TEMPERATURE_FILENAME, sep=tools.SEP, header=None,
                   names=tools.T_HEADER_NAMES, index_col=[0], parse_dates=[0],
                   date_parser=tools.date_parser)

Make sure that both of the data are sorted

In [ ]:
v_df = v_df.sort_values(by='timestamp')
t_df = t_df.sort_values(by='timestamp')

Determine some timestamp characterstics - start time, end time and time horizon of signals

In [ ]:
timestamp_chars = {
    'vibration': {
        'start': v_df.index.to_series()[0],
        'end': v_df.index.to_series()[-1],
        'horizon': v_df.index.to_series()[-1] - v_df.index.to_series()[0]
    },
    'temperature': {
        'start': t_df.index.to_series()[0],
        'end': t_df.index.to_series()[-1],
        'horizon': t_df.index.to_series()[-1] - v_df.index.to_series()[0]
    }
}

print("Timestamp characteristics:")
print(timestamp_chars)

Calculate signal timestamp differences between samples and some statistics about it

In [ ]:
v_timestamp_diff = v_df.index.to_series().diff()
t_timestamp_diff = t_df.index.to_series().diff()

timestamp_diff = pd.concat([v_timestamp_diff, t_timestamp_diff])
timedelta_value_counts = timestamp_diff.value_counts()

print('Timestamp differences:')
print(timedelta_value_counts)

timedelta_statistics = {
    'mean': timestamp_diff.mean(),
    'median': timestamp_diff.median(),
    'mode': timestamp_diff.mode(),
    'max': timestamp_diff.max(),
    'min': timestamp_diff.min()
}

print('Timestamp differences statistics:')
print(timedelta_statistics)

Deduce sampling time with timedelta as the median from the diffs

In [ ]:
sampling_time = timedelta_statistics['median']

print("Median Timedelta that we choose as sampling time: {}".format(sampling_time))

Resample the data to use deduced sampling rate

In [ ]:
v_df = v_df.resample(sampling_time).mean()
t_df = t_df.resample(sampling_time).mean()

Remove outliers using hampel filter with remembering indices of outliers

In [ ]:
USE_HAMPEL = False

if USE_HAMPEL:
    hampel_args = {
        'window_size': 50,
        'n': 3
    }

    v_outliers = hampel.hampel(v_df.squeeze(), **hampel_args)
    t_outliers = hampel.hampel(t_df.squeeze(), **hampel_args)
    print("Number of outliers in data: vibration: "
        "{}%, temperature: {}%".format(len(v_outliers)*100/v_df.shape[0],
                                        len(t_outliers)*100/t_df.shape[0]))
    v_df[v_df.columns[0]] = hampel.hampel(v_df.squeeze(), **hampel_args, imputation=True)
    t_df[t_df.columns[0]] = hampel.hampel(t_df.squeeze(), **hampel_args, imputation=True)

Interpolate the signals

In [ ]:
USE_INTERPOLATION = True

if USE_INTERPOLATION:
    interp_args = {
        'method': 'linear',
        'limit': int(timedelta(minutes=1) / sampling_time),
        'limit_area': 'inside'
    }
    v_df = v_df.interpolate(**interp_args)
    t_df = t_df.interpolate(**interp_args)

Merge dataframes based on the closest timestamp and limit to smallest length

In [ ]:
merge_tolerance = sampling_time
if len(v_df) > len(t_df):
    df = pd.merge_asof(t_df, v_df, on='timestamp', tolerance=merge_tolerance)
else:
    df = pd.merge_asof(v_df, t_df, on='timestamp', tolerance=merge_tolerance)

Set index after merge operation to timestamp again

In [ ]:
df = df.set_index('timestamp')

Plot the timeseries data

In [ ]:
USE_PLOT_WHOLE_DF = False

if USE_PLOT_WHOLE_DF:
    tools.plot_timeseries_data(df)

Split the data into 1-hour length time horizons

In [ ]:
df_groups = df.groupby([(df.index - df.index[0]).astype('timedelta64[1h]')])
groups_num = len(df_groups)

print("Number of groups: {}".format(groups_num))

Interpolate group timeseries to fill out some missing values

In [ ]:
verbose_names = [15, 1024, 1738]

for name, group in df_groups:
    if name in verbose_names:
        print(name)
        tools.plot_timeseries_data(group, style=None)

Calculate the features for each group (ref. https://www.jvejournals.com/article/21622)

In [ ]:
VALIDITY_THRESHOLD = 0.4

non_valid_counter = 0

names_list = list()
features_list = list()
for name, group in df_groups:
    # Calculate number of NaNs to check validity and igniore group
    nans_count = group.isna().sum().sum()
    invalid_part = (nans_count / (group.shape[0]*group.shape[1]))
    # Skip feature extraction when the percentage of invalid values is too high
    if invalid_part > VALIDITY_THRESHOLD:
        non_valid_counter += 1
        if name in verbose_names:
            print("Group {} ignored on validity check.".format(name))
        continue

    # Drop NaNs in the vibration data
    group.dropna(subset=['vibration'], inplace=True)

    # Calculate velocity signal
    velocity = integrate.cumulative_trapezoid(group['vibration'],
                                              x=group.index.astype(np.int64) / 10**9)
    # Extract features
    features = {
        'temperature': group['temperature'].mean(),
        'acc_rms': np.sqrt(group['vibration'].pow(2).mean()),
        'acc_kurtosis': group['vibration'].kurt(),
        'acc_peak_to_peak': group['vibration'].max() - group['vibration'].min(),
        'vel_rms': np.sqrt(np.square(velocity).mean())
    }
    names_list.append(name)
    features_list.append(features)
    if name in verbose_names:
        print("Group {}: {}".format(name, features))

print("Ignored {} groups due to too many missing values.".format(non_valid_counter))

Create DataFrame from features list

In [ ]:
df_features = pd.DataFrame(data=features_list)

Calculate features importance based on normalized variance (versus mean values)

In [ ]:
feature_importance = (df_features.var() / df_features.mean()).sort_values(ascending=False)
print("Feature importance:\n{}".format(feature_importance))

Plot most important features

In [ ]:
df_features.plot.scatter(x=feature_importance.index[3], y=feature_importance.index[4])

Cluster the data using DBSCAN algorithm

In [ ]:
from sklearn import metrics
from sklearn.cluster import DBSCAN

